In [4]:
from ipynb_settings import *

# NPC

Для реализации NPC нам нужно реализовать логику его передвижения. Делать мы это будем в функции `run_logic`

In [5]:
import pygame as pg
import math

class NPC():
    def __init__(self, game, pos):
        self.x, self.y = pos
        self.game = game
        self.player = game.player
        self.theta = 0
    
    def draw(self):
        pg.draw.circle(self.game.screen, 'red',
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
    
    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)
        
    def run_logic(self):
        #! Сейчас начнем реализовывать
        pass
    
    def update(self):
        self.look()
        self.run_logic()
    
    

Что мы добавим в функцию run_logic?

1. Проверку, видит ли `npc` `player`
	- Функцию `ray_cast_player_npc` мы просто скопируем, потому что там сложный для понимая алгоритм
	- Если видит, запускаем функцию `move`

```python
def run_logic(self):
    self.ray_cast_value = self.ray_cast_player_npc()
    if self.ray_cast_value == True:
        self.move()
```

В функции `move()` мы напишем следующее


```python
def move(self):
	next_pos = self.game.pathfinding.get_path(self.map_pos, self.game.player.map_pos)
	next_x, next_y = next_pos
	self.x = next_x
	self.y = next_y
```

In [6]:
from collections import deque


class PathFinding:
    def __init__(self, game):
        self.game = game
        self.map = game.map.mini_map
        self.ways = [-1, 0], [0, -1], [1, 0], [0, 1], [-1, -1], [1, -1], [1, 1], [-1, 1]
        self.graph = {}
        self.get_graph()

    def get_path(self, start, goal):
        self.visited = self.bfs(start, goal, self.graph)
        path = [goal]
        step = self.visited.get(goal, start)

        while step and step != start:
            path.append(step)
            step = self.visited[step]
        return path[-1]

    def bfs(self, start, goal, graph):
        queue = deque([start])
        visited = {start: None}

        while queue:
            cur_node = queue.popleft()
            if cur_node == goal:
                break
            
            #! Эти строчки нам понадобятся потом
            # next_nodes = graph[cur_node]
            # for next_node in next_nodes:
            #     if next_node not in visited and next_node not in self.game.npc_control.npc_positions:
            #         queue.append(next_node)
            #         visited[next_node] = cur_node
            
        return visited

    def get_next_nodes(self, x, y):
        return [(x + dx, y + dy) for dx, dy in self.ways if (x + dx, y + dy) not in self.game.map.world_map]

    def get_graph(self):
        for y, row in enumerate(self.map):
            for x, col in enumerate(row):
                if not col:
                    self.graph[(x, y)] = self.graph.get((x, y), []) + self.get_next_nodes(x, y)



Теперь нужно поключить этот pathfinging к нашей игре, то есть в файл `main.py`

In [ ]:
import pygame as pg
import sys
from settings import *
from map import *
from player import *
from npc import *
#! Импортируем pathfinding
from path_finding import *


class Game:
    def __init__(self) -> None:
        pg.init()
        self.screen = pg.display.set_mode(RES)
        self.clock = pg.time.Clock()
        self.new_game()

    def new_game(self):
        self.delta_time = self.clock.tick(FPS)
        self.map = Map(game=self)
        self.player = Player(game=self)
        self.npc = NPC(game=self, pos=(8, 5))
        #! Добавляем переменную, которую будем использовать для поиска путей
        self.path_finding = PathFinding(game=self)

    def update(self):
        pg.display.flip()        
        self.delta_time = self.clock.tick(FPS)
        self.player.update()
        self.npc.update()

    def draw(self):
        self.screen.fill('black')
        self.map.draw()
        self.player.draw()
        self.npc.draw()

    def check_event(self):
        for event in pg.event.get():
            if event.type == pg.QUIT or (event.type == pg.KEYDOWN and event.key == pg.K_ESCAPE):
                pg.quit()
                sys.exit()

    def run(self):
        while True:
            self.check_event()
            self.draw()
            self.update()


if __name__ == '__main__':
    game = Game()
    game.run()



Но!!! Нам еще надо реализовать методы получения координат для `Player` и `NPC`

1. В классе `NPC`:
	```python 
    @property
    def map_pos(self):
        return int(self.x), int(self.y)
	```

2. В классе `Player`:
	```python 
	@property
	def pos(self):
	    return self.x, self.y

    @property
    def map_pos(self):
        return int(self.x), int(self.y)
	```



Демонстрация, как работает `@property`

In [5]:
class Test:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def map_pos(self):
        return self.x, self.y
    
        
    def pos(self):
        return self.x, self.y
    
test = Test(x=5, y=7)
    
print(test.pos())
#! казалось бы приходится вызывать со скобочками, нам этого не хочется => используем @property
print(test.map_pos)

(5, 7)
(5, 7)


- Обновленный `Player` выглядит слеюущим образом:

In [8]:
from settings import *
import pygame as pg
import math


class Player:
    def __init__(self, game):
        self.game = game
        self.x, self.y = PLAYER_POS
        self.angle = PLAYER_ANGLE
        
    #! Добавили функцию, которая возвращает положение игрока в мире
    @property
    def pos(self):
        return self.x, self.y

    @property
    def map_pos(self):
        return int(self.x), int(self.y)
    #! -------------------------------------------------------------
    
    def draw(self):
        pg.draw.line(self.game.screen, 'green', (self.x * 100 + 14.5 * math.sin(self.angle), self.y * 100 - 14.5 * math.cos(self.angle)),
                     (self.x * 100 + 35 * math.cos(self.angle),
                     self.y * 100 + 35 * math.sin(self.angle)), 2)
        pg.draw.line(self.game.screen, 'green', (self.x * 100 - 14.5 * math.sin(self.angle), self.y * 100 + 14.5 * math.cos(self.angle)),
                     (self.x * 100 + 35 * math.cos(self.angle),
                     self.y * 100 + 35 * math.sin(self.angle)), 2)

        pg.draw.circle(self.game.screen, 'green',
                       (self.x * 100, self.y * 100), 15)
    
    
    def movement(self):
        sin_a = math.sin(self.angle)
        cos_a = math.cos(self.angle)
        dx, dy = 0, 0
        speed = PLAYER_SPEED * self.game.delta_time
        speed_sin = speed * sin_a
        speed_cos = speed * cos_a

        keys = pg.key.get_pressed()
        num_key_pressed = -1
        if keys[pg.K_w]:
            num_key_pressed += 1
            dx += speed_cos
            dy += speed_sin
        if keys[pg.K_s]:
            num_key_pressed += 1
            dx += -speed_cos
            dy += -speed_sin
        if keys[pg.K_a]:
            self.angle -= PLAYER_ROT_SPEED * self.game.delta_time
        if keys[pg.K_d]:
            self.angle += PLAYER_ROT_SPEED * self.game.delta_time

        self.angle %= math.tau
        
        self.check_wall_collision(dx, dy)
    
    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy
    
    def update(self):
        self.movement()

    
    

- Обновленный `NPC` выглядит следующим образом:

In [10]:
import pygame as pg
import math
from settings import *

class NPC():
    def __init__(self, game, pos):
        self.x, self.y = pos
        self.game = game
        self.player = game.player
        self.theta = 0
    
    #! Добавили функцию, которая возвращает положение npc в мире
    @property
    def map_pos(self):
        return int(self.x), int(self.y)
    
    def draw(self):
        pg.draw.circle(self.game.screen, 'red',
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
    
    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)
        
    
    #! Добавили функцию move для передвижения    
    def move(self):
        next_pos = self.game.path_finding.get_path(self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos
        self.x = next_x
        self.y = next_y
        
    #! Добавили функцию для отслеживания далеко ли игрок или нет
    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for i in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False
        
    #! Дописали run_logic
    def run_logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()
    
    def update(self):
        self.look()
        self.run_logic()

Супер!!! Работает, но двигается резко. Почему?



```python 
    def move(self):
        next_pos = self.game.path_finding.get_path(self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos
        
        #! Добавили три строчки для плавного движения
        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time
        #! ------------------------------------------
	
        #! Изменили на self. + d
        self.x = self.x + dx
        self.y = self.y + dy
```

Обновленный `NPC`:

In [ ]:
import pygame as pg
import math
from settings import *

class NPC():
    def __init__(self, game, pos):
        self.x, self.y = pos
        self.game = game
        self.player = game.player
        self.theta = 0
        #! Добавили SPEED_NPC из настроек
        self.speed = SPEED_NPC
    
    #! Добавили функцию, которая возвращает положение npc в мире
    @property
    def map_pos(self):
        return int(self.x), int(self.y)
    
    def draw(self):
        pg.draw.circle(self.game.screen, 'red',
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
    
    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)
        
    
    #! Добавили функцию move для передвижения    
    def move(self):
        next_pos = self.game.path_finding.get_path(self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos
        
        #! Добавили три строчки для плавного движения
        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time
        #! ------------------------------------------

	    #! Изменили на self. + d
        self.x = self.x + dx
        self.y = self.y + dy
        
    #! Добавили функцию для отслеживания далеко ли игрок или нет
    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for i in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False
        
    #! Дописали run_logic
    def run_logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()
    
    def update(self):
        self.look()
        self.run_logic()

- Теперь добавим проверку на стенки

```python 
    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy

```

Обновленный NPC:

In [ ]:
import pygame as pg
import math
from random import randint
from settings import *

class NPC():
    def __init__(self, game, pos):
        self.x, self.y = pos
        self.game = game
        self.player = game.player
        self.theta = 0
        #! Добавили SPEED_NPC из настроек
        self.speed = SPEED_NPC
    
    #! Добавили функцию, которая возвращает положение npc в мире
    @property
    def map_pos(self):
        return int(self.x), int(self.y)
    
    def draw(self):
        pg.draw.circle(self.game.screen, 'red',
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, 'red', (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
    
    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)
    
    #! Добавили функции на проверку столновений со стенками
    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy
    #! ---------------------------------------------------
    
    #! Добавили функцию move для передвижения    
    def move(self):
        next_pos = self.game.pathfinding.get_path(self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos
        
        #! Добавили три строчки для плавного движения
        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time
        #! ------------------------------------------

	    #! Передали управление передвижение функции check_wall_collision
        self.check_wall_collision(dx=dx, dy=dy)
        
    #! Добавили функцию для отслеживания далеко ли игрок или нет
    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for i in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False
        
    #! Дописали run_logic
    def run_logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()
    
    def update(self):
        self.look()
        self.run_logic()

- Запустим игру

In [ ]:
import pygame as pg
import sys
from settings import *
from map import *
from player import *
from npc import *
#! Импортируем pathfinding
from path_finding import *


class Game:
    def __init__(self) -> None:
        pg.init()
        self.screen = pg.display.set_mode(RES)
        self.clock = pg.time.Clock()
        self.new_game()

    def new_game(self):
        self.delta_time = self.clock.tick(FPS)
        self.map = Map(game=self)
        self.player = Player(game=self)
        self.npc = NPC(game=self, pos=(8, 5))
        #! Добавляем переменную, которую будем использовать для поиска путей
        self.path_finding = PathFinding(game=self)

    def update(self):
        pg.display.flip()
        self.delta_time = self.clock.tick(FPS)
        self.player.update()
        self.npc.update()

    def draw(self):
        self.screen.fill('black')
        self.map.draw()
        self.player.draw()
        self.npc.draw()

    def check_event(self):
        for event in pg.event.get():
            if event.type == pg.QUIT or (event.type == pg.KEYDOWN and event.key == pg.K_ESCAPE):
                pg.quit()
                sys.exit()

    def run(self):
        while True:
            self.check_event()
            self.draw()
            self.update()


if __name__ == '__main__':
    game = Game()
    game.run()
